前回と同じリポジトリ(nlp)にjupyterを格納してください。

In [8]:
!pip install gensim mecab-python3 unidic-lite pandas nltk scikit-learn

## 問題4-1. Bag Of Words (テキスト外), mecabによる品詞分解
* Bag Of Wordsについて調べて理解したことを記載してください。
* 以下のテキストを利用してください。
```
text = """
私は昨日、新しい本を買いました。
その本はとても面白くて、一気に読み終えました。
読書は私の大好きな趣味の一つです。
新しい知識を得られるので、毎日少しずつ本を読むようにしています。
"""
```
* Bag Of Wordsを算出してください。そのためにmecabを用いて'名詞', '動詞', '形容詞'のみを抽出し計測してください。


In [6]:
import MeCab
from collections import Counter

def bag_of_words(text: str):
    # MeCabで形態素解析（unicdic-lite用の設定）
    tagger = MeCab.Tagger()
    node = tagger.parseToNode(text)

    words = []
    while node:
        # 品詞が名詞、動詞、形容詞の場合のみ対象とする
        if node.feature.split(',')[0] in ['名詞', '動詞', '形容詞']:
            words.append(node.surface)
        node = node.next

    # 単語の出現回数をカウント
    bow = Counter(words)
    return bow

# テキストの例
text = """
私は昨日、新しい本を買いました。
その本はとても面白くて、一気に読み終えました。
読書は私の大好きな趣味の一つです。
新しい知識を得られるので、毎日少しずつ本を読むようにしています。
"""

# Bag of Words の計算
result = bag_of_words(text)

print("Bag of Words:")
for word, count in result.items():
    print(f"{word}: {count}")

Bag of Words:
昨日: 1
新しい: 2
本: 3
買い: 1
面白く: 1
一気: 1
読み: 1
終え: 1
読書: 1
趣味: 1
一: 1
知識: 1
得: 1
毎日: 1
読む: 1
し: 1
い: 1


## 問題4-2. N-Gram (テキスト外)
1. N-Gramについて調査してください。
2. 以下の文書でBigramを算出してください。

```
text = "I love natural language processing."
```
3. ヒント: ngram計算には形態素解析で分解する必要があります。
4. ヒント: Bigramはfrom nltk.util import ngramsを利用すると簡単にできます。


* Unigram (1-gram): 単一の単語（または文字）。
    例: "I love NLP" では、Unigram は ["I", "love", "NLP"] です。

* Bigram (2-gram): 連続する2つの単語（または文字）の組み合わせ。
    例: "I love NLP" では、Bigram は ["I love", "love NLP"] です。

* Trigram (3-gram): 連続する3つの単語（または文字）の組み合わせ。
    例: "I love NLP" では、Trigram は ["I love NLP"] です。

In [18]:
import MeCab
from nltk.util import ngrams
from collections import Counter

# MeCabの設定
tagger = MeCab.Tagger()  # 分かち書き設定
node = tagger.parseToNode(text)

# サンプルの日本語テキスト
text = "自然言語処理はとても面白い分野です。多くの応用があり、可能性は無限です。"

# トークン化
tokens = []
while node:
    # 品詞が名詞、動詞、形容詞の場合のみ対象とする
    tokens.append(node.surface)
    node = node.next
print("トークン:", tokens)

# N-gram の生成 (例: Bigram)
bigrams = list(ngrams(tokens, 2))
print("Bigrams:", bigrams)

# 頻出N-gramをカウントする
bigram_counts = Counter(bigrams)

# 頻出2-gramの表示
print("\n頻出Bigrams:")
for bigram, count in bigram_counts.most_common():
    print(f"{bigram}: {count}")


トークン: ['', '自然', '言語', '処理', 'は', 'とても', '面白い', '分野', 'です', '。', '多く', 'の', '応用', 'が', 'あり', '、', '可能', '性', 'は', '無限', 'です', '。', '']
Bigrams: [('', '自然'), ('自然', '言語'), ('言語', '処理'), ('処理', 'は'), ('は', 'とても'), ('とても', '面白い'), ('面白い', '分野'), ('分野', 'です'), ('です', '。'), ('。', '多く'), ('多く', 'の'), ('の', '応用'), ('応用', 'が'), ('が', 'あり'), ('あり', '、'), ('、', '可能'), ('可能', '性'), ('性', 'は'), ('は', '無限'), ('無限', 'です'), ('です', '。'), ('。', '')]

頻出Bigrams:
('です', '。'): 2
('', '自然'): 1
('自然', '言語'): 1
('言語', '処理'): 1
('処理', 'は'): 1
('は', 'とても'): 1
('とても', '面白い'): 1
('面白い', '分野'): 1
('分野', 'です'): 1
('。', '多く'): 1
('多く', 'の'): 1
('の', '応用'): 1
('応用', 'が'): 1
('が', 'あり'): 1
('あり', '、'): 1
('、', '可能'): 1
('可能', '性'): 1
('性', 'は'): 1
('は', '無限'): 1
('無限', 'です'): 1
('。', ''): 1


## 問題4-3. 前処理・形態素解析・Word2Vec・cos類似度 (応用)
1. 以下のサンプルデータを用いたdataframeを作成してください。
```
# サンプルの日本語製品レビューのデータ
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
df = pd.DataFrame(data)
}
```
1. レビュー箇所の前処理としてdef preprocess_text(text):を定義し実行してください。基本的に形態素解析した結果を別カラムに入れる想定です。
1. Word2Vecモデルの学習を実施してください。
1. 各レビューの文書ベクトルを計算し、dataframeの"vector"カラムに入れてください。
1. (応用) 文書ベクトルのリストを取り出してコサイン類似度を計算してください。その結果をindex=df['review_id'], columns=df['review_id']のdataframeに格納してください。


In [19]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import MeCab
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 1. CSVファイルの作成（サンプルデータ）
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
}

# データフレームの作成と保存
df = pd.DataFrame(data)

# 2. 前処理関数の定義（形態素解析とトークン化）
def preprocess_text(text):
    mecab = MeCab.Tagger('-Owakati')  # 分かち書きオプションを使用
    text = text.lower()  # 小文字化（日本語には意味が薄いが、念のため）
    text = mecab.parse(text)  # 形態素解析
    tokens = text.split()  # 分かち書きされたテキストをトークンに分割
    return tokens

# 3. データの読み込みと前処理
df['tokens'] = df['review_text'].apply(preprocess_text)

# 4. Word2Vecモデルのトレーニング
model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=1, workers=4)

# 5. 各レビューの文書ベクトルを計算
def document_vector(tokens, model):
    valid_words = [word for word in tokens if word in model.wv]
    if not valid_words:  # 有効な単語がない場合
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word in valid_words], axis=0)

# 各レビューの文書ベクトルを計算してデータフレームに追加
df['doc_vector'] = df['tokens'].apply(lambda tokens: document_vector(tokens, model))

# 6. レビュー間の類似度を計算
# 文書ベクトルのリストを取り出してコサイン類似度を計算
doc_vectors = np.array(df['doc_vector'].tolist())
cosine_similarities = cosine_similarity(doc_vectors)

# 類似度の結果を確認
similarity_df = pd.DataFrame(cosine_similarities, index=df['review_id'], columns=df['review_id'])
print("レビュー間の類似度マトリックス:")
display(similarity_df)


レビュー間の類似度マトリックス:


review_id,1,2,3,4,5
review_id,,,,,
1,1.000000,0.519225,0.366103,0.458701,0.348396
2,0.519225,1.000000,0.527126,0.518637,0.406931
3,0.366103,0.527126,1.000000,0.477087,0.245734
4,0.458701,0.518637,0.477087,1.000000,0.326138
5,0.348396,0.406931,0.245734,0.326138,1.000000


#問題4-4. TF-IDF
1. 以下のデータを利用してdataframeを作成してください。


```
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
}

# データフレームの作成と保存
df = pd.DataFrame(data)
```
1. このdataframeを用いて各レビューでのtf-idfを算出してdataframeの別カラムに格納してください。
1. tf-idfのdataframeを作成してください。index=review_id, カラム: 分割した単語, 値: tf-idf値


In [27]:
import pandas as pd
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer

# サンプルデータ
data = {
    'review_id': [1, 2, 3, 4, 5],
    'review_text': [
        "この商品が大好きです。とても使いやすいです。",
        "最悪の商品です。絶対にお勧めしません。",
        "価格の割に良い品質です。満足しています。",
        "もう一度購入したいと思います。とても良いです。",
        "耐久性がありません。すぐに壊れてしまいました。"
    ]
}

# データフレームの作成
df = pd.DataFrame(data)

# MeCabを使用してテキストをトークン化する関数
# なお、現状は原型を取得しないコードとしている（本来は原型を取得すべき）
def tokenize(text):
    mecab = MeCab.Tagger('-Owakati')  # 分かち書きオプションを使用
    tokens = mecab.parse(text).strip().split() # strip: 空白除去
    return ' '.join(tokens)  # トークンをスペースで結合して一つの文字列にする

# 各レビューのテキストをトークン化
df['tokenized_text'] = df['review_text'].apply(tokenize)

# TF-IDFの計算
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['tokenized_text'])

# 各単語のTF-IDFスコアを確認するためのデータフレームを作成
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=df['review_id'])

# TF-IDFの結果を元のデータフレームに追加（例として、各レビューのTF-IDFベクトルを追加）
df['tfidf_vector'] = list(tfidf_matrix.toarray())

# 表示
print("TF-IDFの結果:")
display(tfidf_df)
print("\n元のデータフレーム:")
display(df)


TF-IDFの結果:


,あり,この,しまい,すぐ,たい,です,とても,まし,ます,ませ,...,商品,壊れ,大好き,思い,最悪,満足,絶対,耐久,良い,購入
review_id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.390095,0.000000,0.000000,0.000000,0.439545,0.314726,0.000000,0.000000,0.000000,...,0.314726,0.000000,0.390095,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.262131,0.000000,0.000000,0.000000,0.375386,...,0.375386,0.000000,0.000000,0.000000,0.465281,0.000000,0.465281,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.262131,0.000000,0.000000,0.375386,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.465281,0.000000,0.000000,0.375386,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.399357,0.224991,0.322199,0.000000,0.322199,0.000000,...,0.000000,0.000000,0.000000,0.399357,0.000000,0.000000,0.000000,0.000000,0.322199,0.399357
5,0.387757,0.000000,0.387757,0.387757,0.000000,0.000000,0.000000,0.387757,0.000000,0.312840,...,0.000000,0.387757,0.000000,0.000000,0.000000,0.000000,0.000000,0.387757,0.000000,0.000000



元のデータフレーム:


,review_id,review_text,tokenized_text,tfidf_vector
0,1,この商品が大好きです。とても使いやすいです。,この 商品 が 大好き です 。 とても 使い やすい です 。,"[0.0, 0.3900946535412576, 0.0, 0.0, 0.0, 0.439..."
1,2,最悪の商品です。絶対にお勧めしません。,最悪 の 商品 です 。 絶対 に お 勧め し ませ ん 。,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.26213107330293306,..."
2,3,価格の割に良い品質です。満足しています。,価格 の 割 に 良い 品質 です 。 満足 し て い ます 。,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.26213107330293306,..."
3,4,もう一度購入したいと思います。とても良いです。,もう 一 度 購入 し たい と 思い ます 。 とても 良い です 。,"[0.0, 0.0, 0.0, 0.0, 0.39935681401701395, 0.22..."
4,5,耐久性がありません。すぐに壊れてしまいました。,耐久 性 が あり ませ ん 。 すぐ に 壊れ て しまい まし た 。,"[0.38775666010579296, 0.0, 0.38775666010579296..."
